In [23]:
from keras.models import model_from_json
import matplotlib.pyplot as plt
from keras.preprocessing.image import img_to_array, load_img
import numpy as np
import pandas as pd

#csvのファイルパスを記載
csv_filepath = 'filename.csv'

#csvからファイルパスを読み込む
csv_file = pd.read_csv(csv_filepath, encoding="UTF-8")#もしくわUTF-8

#csvに記載しているファイルパス、ラベル、総数を格納
fnames = csv_file['filepath']
labels = csv_file['label']
fnames_total = len(fnames)

#データセットを格納する変数の初期化
hoge = np.zeros((fnames_total,32,32,3))

#データセットの格納
#前処理する場合はここでやる
for i,fname in enumerate(fnames):
    temp_img = load_img(fname, target_size=(32,32))

    #画像を配列に変換し0-1で正規化
    temp_img_array = img_to_array(temp_img)
    temp_img_array = temp_img_array.astype('float32')/255.0
    hoge[i] = temp_img_array

#表示
print(fnames)
print(labels)
print(fnames_total)
print("-----------------------")


0     .\\animeface-character-dataset\\thumb\\000_hat...
1     .\\animeface-character-dataset\\thumb\\000_hat...
2     .\\animeface-character-dataset\\thumb\\000_hat...
3     .\\animeface-character-dataset\\thumb\\000_hat...
4     .\\animeface-character-dataset\\thumb\\000_hat...
5     .\\animeface-character-dataset\\thumb\\000_hat...
6     .\\animeface-character-dataset\\thumb\\000_hat...
7     .\\animeface-character-dataset\\thumb\\000_hat...
8     .\\animeface-character-dataset\\thumb\\000_hat...
9     .\\animeface-character-dataset\\thumb\\000_hat...
10    .\\animeface-character-dataset\\thumb\\000_hat...
11    .\\animeface-character-dataset\\thumb\\000_hat...
12    .\\animeface-character-dataset\\thumb\\000_hat...
13    .\\animeface-character-dataset\\thumb\\000_hat...
14    .\\animeface-character-dataset\\thumb\\000_hat...
15    .\\animeface-character-dataset\\thumb\\000_hat...
16    .\\animeface-character-dataset\\thumb\\000_hat...
17    .\\animeface-character-dataset\\thumb\\000